In [ ]:
# read T5 scores for each seg
import collections
import math
from tqdm import tqdm
scores = collections.defaultdict(dict)
# Merge all passage scores into a list of scores
with open('./d2qseg.dev.tsv') as f_query_run_ids, open(
        './mono_preds_d2qseg.dev.t5') as f_pred:
    for line_query_passage_id, line_pred in tqdm(zip(f_query_run_ids, f_pred)):
        query_id, passage_id, _ = line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        _, score = line_pred.strip().split('\t')
        score = float(score)
        scores[query_id][docseg_id] = score

51893214it [02:35, 333813.07it/s]


In [ ]:
#get T5 for each seg for bm25+d2q top1000 and take maxP for each doc and reranked 
examples = collections.defaultdict(dict)
with open('./run.doc.dev.1000.small.tsv') as fcan, open('./run.monoT5.doc.1000.trec', 'w') as fout:
    for line_query_passage_id in tqdm(fcan):
        query_id, passage_id, _ = line_query_passage_id.strip().split('\t')
        docseg_id = passage_id.split('.')[0]
        doc_id = docseg_id.split('#')[0]
        score = scores[query_id][docseg_id]
        score = float(score)
        if doc_id not in examples[query_id]:
            examples[query_id][doc_id] = [score]
        else:
            examples[query_id][doc_id].append(score)
    
    for query_id, doc_ids_scores in tqdm(examples.items()):
        doc_ids_scores = [
            (doc_id, max(scores))
            for doc_id, scores in doc_ids_scores.items()]

        doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
        rank = 1
        for doc_id, score in doc_ids_scores:
            score = math.exp(score)
            # Adds small score based on rank to avoid duplicated scores.
            score += 1e-6 / rank
            fout.write(f'{query_id} Q0 {doc_id} {rank} {score} T5\n')
            rank += 1

5191985it [00:13, 394191.20it/s]
100%|██████████| 5193/5193 [00:05<00:00, 865.89it/s]


In [ ]:
#only takes top 100 for each doc for msmarco_doc_eval.py
with open ('./run.monoT5.doc.1000.tsv', 'w') as fout:
    for query_id, doc_ids_scores in tqdm(examples.items()):
            doc_ids_scores = [
                (doc_id, max(scores))
                for doc_id, scores in doc_ids_scores.items()]

            doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
            rank = 1
            for doc_id, score in doc_ids_scores:
                if (rank < 100):
                    score = math.exp(score)
            # Adds small score based on rank to avoid duplicated scores.
                    score += 1e-6 / rank
                    fout.write(f'{query_id}\t{doc_id}\t{rank}\n')
                rank += 1

100%|██████████| 5193/5193 [00:02<00:00, 2455.47it/s]


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
 --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run ./run.monoT5.doc.1000.tsv

Quantity of Documents ranked for each query is as expected. Evaluating
#####################
MRR @100: 0.40878809930075677
QueriesRanked: 5193
#####################


In [ ]:
!python3 ../RA/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_run.py --input ./run.monoT5.doc.1000.tsv --output ./run.monoT5.doc.1000.trec

Done!


In [ ]:
!../RA/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mmap -m ndcg_cut.10\
 ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt ./run.monoT5.doc.1000.trec

map                   	all	0.4088
ndcg_cut_10           	all	0.4760


In [ ]:
for ltr_k in [100,200,300,400,500,600,700,800,900]:
    for k in [1000]:
        counts = None
        counts = collections.defaultdict(int)
        with open(f'run.doc.dev.{ltr_k}from{k}.tsv', 'w') as output_f:
            with open(f'run.doc_pas.dev.d2q.1000.small.tsv') as input_f:
                for line in tqdm(input_f):
                    cols = line.split('\t')
                    qid = cols[0]
                    if(counts[qid] < ltr_k):
                        docid = cols[1]
                        counts[qid] += 1
                        output_f.write(f'{qid}\t{docid}\t{counts[qid]}\n')

5191985it [00:05, 881457.24it/s] 
5191985it [00:06, 824838.94it/s]
5191985it [00:10, 515200.70it/s]
5191985it [00:11, 456653.82it/s]
5191985it [00:11, 440796.72it/s]
5191985it [00:13, 397367.77it/s]
5191985it [00:12, 399907.77it/s]
5191985it [00:10, 510947.22it/s]
5191985it [00:12, 400384.58it/s]


In [ ]:
#get T5 for each seg for bm25+d2q top1000 and take maxP for each doc and reranked 

for ltr_k in [100,200,300,400,500,600,700,800,900]:
    for k in [1000]:
        examples = None
        examples = collections.defaultdict(dict)
        with open(f'./run.doc.dev.{ltr_k}from{k}.tsv') as fcan, open(f'./run.monoT5.doc.{ltr_k}from1000.d2q_baseline.tsv', 'w') as fout:
            for line_query_passage_id in tqdm(fcan):
                query_id, passage_id, _ = line_query_passage_id.strip().split('\t')
                docseg_id = passage_id.split('.')[0]
                doc_id = docseg_id.split('#')[0]
                if docseg_id in scores[query_id]:
                    score = scores[query_id][docseg_id]
                    score = float(score)
                    if doc_id not in examples[query_id]:
                        examples[query_id][doc_id] = [score]
                    else:
                        examples[query_id][doc_id].append(score)
    
            for query_id, doc_ids_scores in tqdm(examples.items()):
                doc_ids_scores = [
                    (doc_id, max(scores))
                    for doc_id, scores in doc_ids_scores.items()]

                doc_ids_scores.sort(key=lambda x: x[1], reverse=True)
                rank = 1
                for doc_id, score in doc_ids_scores:
                    score = math.exp(score)
                    # Adds small score based on rank to avoid duplicated scores.
                    score += 1e-6 / rank
                    fout.write(f'{query_id}\t{doc_id}\t{rank}\n')
                    rank += 1

519300it [00:08, 60063.90it/s]
100%|██████████| 5193/5193 [00:00<00:00, 11534.36it/s]
1038600it [00:04, 259580.31it/s]
100%|██████████| 5193/5193 [00:00<00:00, 5824.26it/s]
1557900it [00:06, 248778.58it/s]
100%|██████████| 5193/5193 [00:01<00:00, 3485.43it/s]
2077126it [00:08, 235358.17it/s]
100%|██████████| 5193/5193 [00:01<00:00, 2807.06it/s]
2596326it [00:10, 255509.14it/s]
100%|██████████| 5193/5193 [00:01<00:00, 2622.48it/s]
3115526it [00:11, 280354.18it/s]
100%|██████████| 5193/5193 [00:02<00:00, 2043.94it/s]
3634685it [00:12, 291312.89it/s]
100%|██████████| 5193/5193 [00:02<00:00, 2262.26it/s]
4153785it [00:15, 264876.21it/s]
100%|██████████| 5193/5193 [00:02<00:00, 1962.38it/s]
4672885it [00:18, 254447.30it/s]
100%|██████████| 5193/5193 [00:03<00:00, 1468.33it/s]


In [ ]:

import os
for ltr_k in [100,200,300,400,500,600,700,800,900]:
    for k in [1000]:
        stream = os.popen(f"python3 ../RA/pyserini/tools/scripts/msmarco/msmarco_doc_eval.py \
         --judgments ../RA/pyserini/tools/topics-and-qrels/qrels.msmarco-doc.dev.txt --run run.monoT5.doc.{ltr_k}from1000.d2q_baseline.tsv")
        output = stream.read()
        outputs = output.split('Quantity of Documents ranked for each query is as expected.')
        print(f'{ltr_k} from {k}:')
        print(outputs[1])

100 from 1000:
 Evaluating
#####################
MRR @100: 0.399996988532227
QueriesRanked: 5193
#####################

200 from 1000:
 Evaluating
#####################
MRR @100: 0.4050033080597611
QueriesRanked: 5193
#####################

300 from 1000:
 Evaluating
#####################
MRR @100: 0.40691860737864943
QueriesRanked: 5193
#####################

400 from 1000:
 Evaluating
#####################
MRR @100: 0.4074320235975577
QueriesRanked: 5193
#####################

500 from 1000:
 Evaluating
#####################
MRR @100: 0.4085256650058111
QueriesRanked: 5193
#####################

600 from 1000:
 Evaluating
#####################
MRR @100: 0.4087358364737934
QueriesRanked: 5193
#####################

700 from 1000:
 Evaluating
#####################
MRR @100: 0.4092244804119087
QueriesRanked: 5193
#####################

800 from 1000:
 Evaluating
#####################
MRR @100: 0.40896091715694155
QueriesRanked: 5193
#####################

900 from 1000:
 Evaluating
####